In [2]:
import torch
import pytorch_lightning as pl 


In [3]:
import os
# import cv2
import random

import numpy as np
import matplotlib.pyplot as plt

import albumentations as A

from albumentations.pytorch import ToTensorV2

ImportError: libGL.so.1: cannot open shared object file: No such file or directory

In [4]:
# function for visualization
def visualize(**images):
    n = len(images)
    plt.figure(figsize=(16,5))
    for i, (name, image) in enumerate(images.items()):
        plt.subplot(1, n, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.title(' '.join(name.split('_')).title())
        # plt.imshow(image, cmap='gray', vmin=0, vmax=255)
        plt.imshow(image)
    plt.show()

In [15]:
from libtiff import TIFF
from libtiff import TIFFfile, TIFFimage
import glob
# import cv2
%matplotlib inline
from skimage import io, img_as_ubyte

In [7]:
import re
numbers = re.compile(r'(\d+)')
def numericalSort(value):
    parts = numbers.split(value)
    parts[1::2] = map(int, parts[1::2])
    return parts
# Train dataset
filelist_trainx = sorted(glob.glob('/root/eye-in-the-sky/Inter-IIT-CSRE/The-Eye-in-the-Sky-dataset/sat/*.tif'), key=numericalSort)
# Train label
filelist_trainy = sorted(glob.glob('/root/eye-in-the-sky/Inter-IIT-CSRE/The-Eye-in-the-Sky-dataset/gt/*.tif'), key=numericalSort)
# Test dataset
filelist_testx = sorted(glob.glob('/root/eye-in-the-sky/Inter-IIT-CSRE/The-Eye-in-the-Sky-dataset/sat_test/*.tif'), key=numericalSort)

In [16]:
trainx_list = []
for fname in filelist_trainx[:13]:
    # tif = TIFF.open(fname)
    tif = io.imread(fname)
    # image = tif.read_image()
    print(tif.shape)
    # TIFF.write_image(image)
    # tif.close()
    # img = cv2.imread(fname, 1)
    # print(img)

    # break

(1295, 1364, 4)
(926, 943, 4)
(1107, 1182, 4)
(1639, 1672, 4)
(1342, 1447, 4)
(786, 910, 4)
(957, 1050, 4)
(1146, 872, 4)
(748, 800, 4)
(1066, 1491, 4)
(1001, 1068, 4)
(812, 984, 4)
(1476, 1524, 4)


In [9]:
# Resizing the image to nearest dimensions multipls of 'stride'

def resize(img, stride, n_h, n_w):
    #h,l,_ = img.shape
    ne_h = (n_h*stride) + stride
    ne_w = (n_w*stride) + stride
    
    img_resized = imresize(img, (ne_h,ne_w))
    return img_resized

In [10]:
# Padding at the bottem and at the left of images to be able to crop them into 128*128 images for training

def padding(img, w, h, c, crop_size, stride, n_h, n_w):
    
    w_extra = w - ((n_w-1)*stride)
    w_toadd = crop_size - w_extra
    
    h_extra = h - ((n_h-1)*stride)
    h_toadd = crop_size - h_extra
    
    img_pad = np.zeros(((h+h_toadd), (w+w_toadd), c))
    #img_pad[:h, :w,:] = img
    #img_pad = img_pad+img
    img_pad = np.pad(img, [(0, h_toadd), (0, w_toadd), (0,0)], mode='constant')
    
    return img_pad

In [11]:
# Adding pixels to make the image with shape in multiples of stride

def add_pixals(img, h, w, c, n_h, n_w, crop_size, stride):
        
    w_extra = w - ((n_w-1)*stride)
    w_toadd = crop_size - w_extra

    h_extra = h - ((n_h-1)*stride)
    h_toadd = crop_size - h_extra

    img_add = np.zeros(((h+h_toadd), (w+w_toadd), c))
        
    img_add[:h, :w,:] = img
    img_add[h:, :w,:] = img[:h_toadd,:, :]
    img_add[:h,w:,:] = img[:,:w_toadd,:]
    img_add[h:,w:,:] = img[h-h_toadd:h,w-w_toadd:w,:]
            
    return img_add  

In [12]:
# Slicing the image into crop_size*crop_size crops with a stride of crop_size/2 and makking list out of them

def crops(a, crop_size = 128):
    
    #stride = int(crop_size/2)
    stride = 32

    croped_images = []
    h, w, c = a.shape
    
    n_h = int(int(h/stride))
    n_w = int(int(w/stride))
    
    # Padding using the padding function we wrote
    a = padding(a, w, h, c, crop_size, stride, n_h, n_w)
    
    # Resizing as required
    ##a = resize(a, stride, n_h, n_w)
    
    # Adding pixals as required
    #a = add_pixals(a, h, w, c, n_h, n_w, crop_size, stride)
    
    # Slicing the image into 128*128 crops with a stride of 64
    for i in range(n_h-1):
        for j in range(n_w-1):
            crop_x = a[(i*stride):((i*stride)+crop_size), (j*stride):((j*stride)+crop_size), :]
            croped_images.append(crop_x)
    return croped_images

In [18]:
'''
tif图像和标签crop成128×128的块，用来训练，stride=32
'''
trainx_list = []

for fname in filelist_trainx[:13]:
    
    # Reading the image
    tif = TIFF.open(fname)
    image = tif.read_image()
    
    # Padding as required and cropping
    crops_list = crops(image)
    #print(len(crops_list))
    trainx_list = trainx_list + crops_list
    
# Array of all the cropped Training sat Images    
trainx = np.asarray(trainx_list)


# Reading, padding, cropping and making array of all the cropped images of all the trainig gt images
trainy_list = []

for fname in filelist_trainy[:13]:
    
    # Reading the image
    # tif = TIFF.open(fname)
    # image = tif.read_image()
    image = io.imread(fname)
    
    # Padding as required and cropping
    crops_list =crops(image)
    
    trainy_list = trainy_list + crops_list
    
# Array of all the cropped Training gt Images    
trainy = np.asarray(trainy_list)


# Reading, padding, cropping and making array of all the cropped images of all the testing sat images
testx_list = []

#for fname in filelist_trainx[13]:
    
    # Reading the image
# tif = TIFF.open(filelist_trainx[13])
# image = tif.read_image()
image = io.imread(filelist_trainx[13])
    
# Padding as required and cropping
crops_list = crops(image)
    
testx_list = testx_list + crops_list
    
# Array of all the cropped Testing sat Images  
testx = np.asarray(testx_list)


# Reading, padding, cropping and making array of all the cropped images of all the testing sat images
testy_list = []

#for fname in filelist_trainx[13]:
    
# Reading the image
# tif = TIFF.open(filelist_trainy[13])
# image = tif.read_image()
image = io.imread(filelist_trainy[13])
    
# Padding as required and cropping
crops_list = crops(image)
    
testy_list = testy_list + crops_list
    
# Array of all the cropped Testing sat Images  
testy = np.asarray(testy_list)

'''
原始图像不进行crop
'''
# Making array of all the training sat images as it is without any cropping

xtrain_list = []

for fname in filelist_trainx:
    
    # Reading the image
    # tif = TIFF.open(fname)
    # image = tif.read_image()
    image = io.imread(fname)
    
    crop_size = 128
    
    stride = 64
    
    h, w, c = image.shape
    
    n_h = int(int(h/stride))
    n_w = int(int(w/stride))
    
    
    image = padding(image, w, h, c, crop_size, stride, n_h, n_w)
    
    xtrain_list.append(image)

x_train = np.asarray(xtrain_list)
# tif = TIFF.open('Inter-IIT-CSRE/The-Eye-in-the-Sky-dataset/sat/14.tif')
# image = tif.read_image()
image = io.imread('/root/dataset/The-Eye-in-the-Sky-dataset/sat/14.tif')
crop_size = 128
    
stride = 64
h, w, c = image.shape
    
n_h = int(int(h/stride))
n_w = int(int(w/stride))
    
    
image = padding(image, w, h, c, crop_size, stride, n_h, n_w)
x_train = image
# Making array of all the training gt images as it is without any cropping

ytrain_list = []

for fname in filelist_trainy:
    
    # Reading the image
    # tif = TIFF.open(fname)
    # image = tif.read_image()
    image = io.imread(fname)
    
    crop_size = 128
    
    stride = 64
    
    h, w, c = image.shape
    
    n_h = int(int(h/stride))
    n_w = int(int(w/stride))
    
    
    image = padding(image, w, h, c, crop_size, stride, n_h, n_w)
    
    ytrain_list.append(image)

y_train = np.asarray(ytrain_list)


# tif = TIFF.open('Inter-IIT-CSRE/The-Eye-in-the-Sky-dataset/gt/14.tif')
# image = tif.read_image()
image = io.imread('/root/dataset/The-Eye-in-the-Sky-dataset/gt/14.tif')
crop_size = 128
    
stride = 64
    
h, w, c = image.shape
    
n_h = int(int(h/stride))
n_w = int(int(w/stride))
    
    
image = padding(image, w, h, c, crop_size, stride, n_h, n_w)
y_train = image

In [37]:
# print(trainx.shape)
# print(trainy.shape)
# print(testx.shape)
# print(testy.shape)
# print(trainx.dtype)
print(x_train.shape)
img = x_train[:,:,:3]
img.shape
image = Image.fromarray(img, 'RGB')
image.save('image1.png')

(640, 832, 4)


In [31]:
from PIL import Image
image = Image.fromarray(trainy[1], 'RGB')
image.save('train1.png')
image.show()

In [46]:
from torch.utils.data import Dataset
class CellDataset(Dataset):
    def __init__(self, trainx, trainy, testx, testy, eval=False):
        if(eval is False):
            self.trainx = trainx
            self.trainy = trainy
        else:
            self.trainx = testx
            self.trainy = testy
    def __len__(self):
        return len(self.trainx)
    def __getitem__(self, idx):
        image = torch.tensor(self.trainx[idx].astype(np.int16), dtype=torch.float32)
        label = torch.from_numpy(self.trainy[idx])
        image = image.float().unsqueeze(0).unsqueeze(0)
        return image, label


In [51]:
dataset = CellDataset(trainx, trainy, testx, testy, False)
print(dataset[0][0].shape)
print(dataset[0][1].shape)

torch.Size([1, 1, 128, 128, 4])
torch.Size([128, 128, 3])
